# <div align="center">PIFu: Pixel-Aligned Implicit Function for High-Resolution Clothed Human Digitization</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)


***PIFu is a memory efficient and spatially aligned 3D representation for 3D surfaces.*** An implicit function defines a surface as a level set of a function f, e.g. $f(X) = 0$. 

> In mathematics, a ***level set*** of a real-valued function f of n real variables is a set of the form $L_c(f) = \{ (x_1, x_2, ..., x_n) | f(x_1, x_2, ..., x_n) = c \}$, that is, a set where the function takes on a given constant value c. When the number of variables is two, a level set is generically a curve, called a ***level curve***, ***contour line***, or ***isoline***. So a level curve is the set of all real-valued solutions of an equation in two variables x1 and x2. When n = 3, a level set is called a ***level surface*** or ***isosurface***, and for higher values of n the level set is a level ***hypersurface***. So a level surface is the set of all real-valued roots of an equation in three variables x1, x2 and x3, and a level hypersurface is the set of all real-valued roots of an equation in n (n > 3) variables.

The proposed pixel-aligned implicit function consists of a ***fully convolutional image encoder g*** and a ***continuous
implicit function f represented by multi-layer perceptrons (MLPs)***, where the surface is defined as a level set of

<div align="center">$f(F(x), z(X)) = s : s \in R \tag{1}$</div>

where 

* for 3D point $X, x = \pi(X)$ is its 2D projection, 
 
 
* $z(X)$ is the depth value in the ***camera coordinate space***, 
 
 
* $F(x) = g(I(x))$ is the image feature at x

> When you take a photograph, you need to move and rotate the camera to adjust the viewpoint. So in a way, when you transform a camera (by translating and rotating it — note that scaling a camera doesn't make much sense), what you actually do is transforming a local coordinate system which implicitly represents the transformations applied to that camera. In CG (computer graphics) we call this a ***camera coordinate*** system. <img src='assets/1.png'> A coordinate system $[i, j, k]$ centered at the pinhole $O$ such that the axis $k$ is perpendicular to the image plane and points toward it. 

We assume a ***weak-perspective camera***, but extending to perspective cameras is straightforward. Note that we obtain the pixelaligned feature F(x) using ***bilinear sampling***, because the 2D projection of X is defined in a continuous space rather
than a discrete one (i.e., pixel).

> ***3D projection*** or graphical projection maps points in 3D onto a 2D. We can represent the relationship between a point $P$ in 3D space and its $P'$ image coordinates by a matrix vector relationship: $P' = \begin{bmatrix}
       \alpha & 0 & c_x & 0     \\[0.5em]
       0 & \beta & c_y &  0     \\[0.5em]
       0 & 0 & 1 &  0           \\[0.5em]
     \end{bmatrix} P = MP = \begin{bmatrix}
       \alpha & 0 & c_x     \\[0.5em]
       0 & \beta & c_y      \\[0.5em]
       0 & 0 & 1           \\[0.5em]
     \end{bmatrix} \;[I,\; 0]\; P = K \;[I,\; 0]\; P$ and  $P = \begin{bmatrix}
       R & T     \\[0.5em]
       0 & 1     \\[0.5em]
     \end{bmatrix} P_w$, so $P'= K \;[R\; T]\; P_w$
     
> In the ***weak perspective model***, points are first projected to the camera cordinate space (reference plane) using orthogonal projection and then projected to the image plane using a projective transformation.

## <div align="center">Single-view Surface Reconstruction</div>
---------------------------------------------------------------------

For surface reconstruction, we represent the ground truth surface as a ***0.5 level-set of a continuous 3D occupancy field***:

<div align="center">$f_v^{*}(X) = \begin{cases} 1, & \mbox{if } X\mbox{ is inside mesh surface} \\ 0, & \mbox{otherwise} \end{cases} \tag{2}$</div>

>

We train a pixel-aligned implicit function (PIFu) $f_v$ by minimizing the average of mean squared error:

<div align="center">$L_V = 1/n \sum_{i=1}^{n}|f_v(F_V(x_i), z(X_i)) - f_v^{*}(X_i)|^2 \tag{3}$</div>

where

* $X_i \in R^3 $ , 


* $F_V(x) = g(I(x))$ is the image feature from the image encoder $g$ at $x = \pi(X)$ and 


* $n$ is the number of sampled points.

Given a pair of an input image and the corresponding 3D mesh that is ***spatially aligned*** with the input image, the parameters of the image encoder $g$ and PIFu $f_v$ are jointly updated by minimizing Eq. 3. 

> In image fusion, ***spatial alignment*** defined as the process of geometrically aligning two or more images of the same
scene acquired at different times (multi-temporal fusion), or with different sensors (multi-modal fusion), or from different viewpoints (multi-view fusion). It is a crucial pre-processing operation in image fusion and its accuracy is a major factor in
determining the quality of the output image. Suppose we have two input images, $A$ and $B$, our task is to find the transformation $T$ which "optimally" maps spatial locations in the image $B$ to the corresponding spatial locations in the image $A$. If $(u,v)$ denotes a pixel location in the reference image $A$ and $(x,y)$ denotes a pixel location in the floating image $B$, then the transformation $T$ represents a mapping of every pixel location $(x,y)$ in $B$ into the corresponding location $(u',v')$ in $A$.

> Let $B'$ be the corresponding tranformed $B$ image. The image $B'$ is only defined at the points $(u',v')$, where by definition, $B'(u',v') = B(x,y)$. In order to convert $B'(u',v')$ into a digital image which is defined at the same pixel locations as $A$ we apply an ***interpolation/resampling*** operation to $B'(u',v')$:

As Bansal demonstrate for semantic segmentation, training an image encoder with a subset of pixels does not hurt convergence compared with training with all the pixels. During inference, we densely sample the probability field over the 3D space and extract the iso-surface of the probability field at threshold 0.5 using the ***Marching Cube*** algorithm. This implicit surface representation is suitable for detailed objects with arbitrary topology. Aside from PIFu’s expressiveness and memoryefficiency, we develop a spatial sampling strategy that is critical for achieving high-fidelity inference.

> The ***Marching cubes algorithm*** can be described as follows: Given an object, a test to determine whether an arbitrary point is within the object, and bounds within which the object exists: Divide the space within the bounds into an arbitrary number of cubes.  Test the corners of every cube for whether they are inside the object.  *For every cube where some corners are inside and some corners are outside the object, the surface must pass through that cube*, intersecting the edges of the cube in between corners of opposite classification.  Draw a surface within each cube connecting these intersections.  You have your object.

> To understand how the Marching Cubes algorithm works, let's take a 2D case and what might be called the ***Marching Squares algorithm***. 

> The object   | Divide into squares    | Red for in, blue for out  | Between inside and outside  | Approximation  |
> :----------------:|:-----------------:|:-----------------:|:-----------------:|:---------------:
> ![](assets/2.jpg) | ![](assets/3.jpg) | ![](assets/4.jpg) | ![](assets/5.jpg) | ![](assets/6.jpg)

> Implementing the algorithm in 3D works much the same as it did in 2D.  For slice data like the Visible Human Male dataset, you stack the slices in 3D, knowing each slice is 1mm or 3 pixels appart.  In order to be able to test the vertices of each cube, you must choose your cube size to align with the slices, either using 1mm cubes (or rectangles 1mm high and 1/3rd of a mm thick since pixels are only 1/3rd of a mm wide), or some multiple of 1mm cubes (ie 2mm or 3 mm) so that each horizontal side of your cubes falls on the plane of a slice.  You then can test each vertex by going to the masked slices corresponding to each cube's top and bottom z values.  You now have a bunch of cubes with labeled corners.  For each cube, you know the surface intersects the cube along the edges in between corners of opposing classifications.  Each cube should look something like this:

> The object   | Divide into squares    |
> :----------------:|:-----------------:
> ![](assets/7.jpg) | ![](assets/8.jpg)

> For more details please follow the video [link](https://www.youtube.com/watch?v=B_xk71YopsA)

### Spatial Sampling

The resolution of the training data plays a central role in achieving the expressiveness and accuracy of our implicit function. Unlike voxel-based methods, our approach does not require discretization of ground truth 3D meshes. Instead, we can directly sample 3D points on the fly from the ground truth mesh in the original resolution using an efficient ***ray tracing algorithm***. Note that this operation requires ***water-tight meshes***. In
the case of non-watertight meshes, one can use off-the-shelf solutions to make the meshes watertight.

> In computer graphics, ***water-tight*** meshes usually describe meshes consisting of one closed surface. In this sense, water-tight meshes ***do not contain holes*** and have a clearly defined inside. Therefore, they are commonly required by many applications in computer graphics as well as in computer vision — for example, when voxelizing meshes into occupancy grids or signed distance functions.

> btw, a ***triangular mesh*** $M=(V,F)$ is defined by a set of vertices $V \subseteq R^3$ and a set of triangular faces $F \subseteq \{1,…,|V|\}^3$ such that $f=(f1,f2,f3) \in F$ defines a triangular face enclosed by the corresponding vertices $v_{f1}$, $v_{f2}$ and $v_{f3}$. Faces implicitly also define the edges  $E(F)$ between the vertices.

Additionally,we observe that the sampling strategy can largely influence the final reconstruction quality. If one uniformly samples points in the 3D space, the majority of points are far from the iso-surface, which would unnecessarily weight the network toward outside predictions. On the other hand, sampling only around the iso-surface can cause overfitting. Consequently, we propose to combine uniform sampling and adaptive sampling based on the surface geometry. We first randomly sample points on the surface geometry and add offsets with normal distribution $N(0, \sigma)$ ($\sigma = 5.0$ cm in our experiments) for x, y, and z axis to perturb their positions around the surface. We combine those samples with uniformly sampled points within bounding boxes using a ratio of $16 : 1$. We provide an ablation study on our sampling strategy in the supplemental materials.

## <div align="center">Texture Inference</div>
---------------------------------------------------------------------

PIFu enables us to directly predict the RGB colors on the surface geometry by defining s in ***Eq. 1*** as an RGB vector field instead of a scalar field. This supports texturing of shapes with arbitrary topology and self-occlusion. However, extending PIFu to color prediction is a non-trivial task as RGB colors are defined only on the surface while the 3D occupancy field is defined over the entire 3D space. Here, we highlight the modification of PIFu in terms of training procedure and network architecture. Given sampled 3D points on the surface $X \in \Omega$, the objective function for texture inference is the average of $L1$ error of the sampled colors as follows:

<div align="center">$L_C = 1/n \sum_{i=1}^{n}|f_c(F_C(x_i), z(X_i)) - C(X_i)| \tag{4}$</div>

where

* $C(X_i)$ is the ground truth RGB value on the surface point $X_i \in \Omega$ and 


* n is the number of sampled points.

We found that naively training $f_c$ with the loss function above severely suffers from overfitting. The problem is that $fc$ is expected to learn not only RGB color on the surface but also the underlining 3D surfaces of the object so that $fc$ can infer texture of unseen surface with different pose and shape during inference, which poses a significant challenge. We address this problem with the following modifications. First, we 

* condition the image encoder for texture inference with the image features learned for the surface reconstruction $F_V$ . This way, the image encoder can focus on color inference of a given geometry even if unseen objects have different shape, pose, or topology. 


* Additionally, we introduce an offset $\epsilon \sim N(0, d)$ to the surface points along the surface normal N so that the color can be defined not only on the exact surface but also on the 3D space around it. 

With the modifications above, the training objective function can be rewritten as:

<div align="center">$L_C = 1/n \sum_{i=1}^{n}|f_c(F_C(x_i^{'}, F_V), X_{i, z}^{'}) - C(X_i)| \tag{5}$</div>

where

* $X_i^{'} = X_i + \epsilon N_i$, We use d = 1.0 cm for all the experiments.

## <div align="center">Data sets</div>
---------------------------------------------------------------------

Since there is no large scale datasets for high-resolution clothed humans, we collected photogrammetry data of 491 high-quality textured human meshes with a wide range of clothing, shapes, and poses, each consisting of about 100, 000 triangles from ***RenderPeople***. We refer to this database as ***HighFidelity Clothed Human Data set***. We randomly split the dataset into a training set of 442 subjects and a test set of 49 subjects. To efficiently render the digital humans, ***Lambertian diffuse shading*** with surface normal and spherical harmonics are typically used due to its simplicity and efficiency. 

However, we found that to achieve high-fidelity reconstructions on real images, the synthetic renderings need to correctly simulate light transport effects resulting from both global and local geometric properties such as ambient occlusion. To this end, we use a ***precomputed radiance transfer*** technique (PRT) that precomputes visibility on the surface using spherical harmonics and efficiently represents global light transport effects by multiplying spherical harmonics coefficients of illumination and visibility. PRT only needs to be computed once per object and can be reused with arbitrary illuminations and camera angles. Together with PRT, we use 163 second-order spherical harmonics of indoor scene from HDRI Haven using random rotations around y axis. We render the
images by aligning subjects to the image center using a weakperspective camera model and image resolution of $512 x 512$. We also rotate the subjects for 360 degrees in yaw axis, resulting in $360 x 442 = 159.120$ images for training. 

For the evaluation, we render 49 subjects from RenderPeople and 5 subjects from the BUFF data set using 4 views spanning every 90 degrees in yaw axis. Note that we render the images without the background. We also test our approach on real images of humans from the DeepFashion data set. In the case of real data, we use a offthe-shelf semantic segmentation network together with ***Grab-Cut*** refinement.